In [19]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

/home/mrzaizai2k/code_Bao/LLM-with-RAG/notebook


In [20]:
from src.Utils.utils import *
from src.ragqa import *

In [21]:
device = take_device()

Selected device: cuda


In [22]:
def remove_repetitive_patterns(text:str):
    # Define regex patterns to match repetitive sequences
    patterns = [
        r'(.\n\n)+',  # Matches repetitive sequences of ".\n"
        r'(\\\(T\\.\\\)\\\(\\.\\\))+',   # Matches repetitive sequences of "\(T.\)\(.\)"
        r'(\w+)\1+',                      # Matches repetitive sequences of a word
        r'(\\!)+',
        r'(\*\*Algorithm \d+\*\*\s*)+',  # Matches "Algorithm" patterns with numbers
    ]
    
    # Process each pattern and replace it with a single instance
    for pattern in patterns:
        text = re.sub(pattern, r'\1', text)
    
    return text

In [23]:
text = """**Algorithm 1** **Algorithm 2** **Algorithm 3** **Algorithm 4** **Algorithm 5** **Algorithm 6** **Algorithm 7** **Algorithm 8** **Algorithm 9** **Algorithm 10** **Algorithm 1** **Algorithm 12** **Algorithm 13** **Algorithm 14** **Algorithm 15** **Algorithm 16*"""
print(remove_repetitive_patterns(text))

**Algorithm 15** **Algorithm 16*


In [24]:
model_config = config_parser(data_config_path = 'config/model_config.yaml')

## Load the vector database

In [31]:
embedding_model = HuggingFaceEmbeddings(
    model_name=model_config["embedding_model"], 
    # model_name="model/distilroberta-triplets_2", #
    # multi_process=True,
    model_kwargs={"device": device},
    # encode_kwargs={"normalize_embeddings": True},  # Set `True` for cosine similarity
)

db_faiss_path = "data/vectorstores/db_faiss_4"
knowledge_vector_db = FAISS.load_local(db_faiss_path, 
                                        embedding_model, 
                                        allow_dangerous_deserialization='True',
                                        )


In [32]:
docs_processed=get_documents_from_vectordb(knowledge_vector_db)
len(docs_processed)

46770

In [27]:
docs_processed[800:805]

[Document(page_content='Al dependencies cros the boundary\nlines pointing inward, folowing the Dependency Rule.\n\nCONCLUSION\nConforming to these simple rules is not dificult, and it wil save you a lot of\nheadaches going forward.\n\nBy separating the software into layers and conforming\nto the Dependency Rule, you wil create a system that is intrinsicaly testable,\nwith al the benefits that implies.', metadata={'source': 'data/web_data/[1]. Clean Architecture A Craftsman Guide to Software Structure and Design.pdf', 'file_path': 'data/web_data/[1]. Clean Architecture A Craftsman Guide to Software Structure and Design.pdf', 'page': 208, 'total_pages': 444, 'format': 'PDF 1.4', 'title': "Clean Architecture: A Craftsman's Guide to Software Structure and Design (Robert C. Martin Series)", 'author': 'Robert C. Martin', 'subject': '', 'keywords': '', 'creator': 'calibre 3.7.0 [https://calibre-ebook.com]', 'producer': 'calibre 3.7.0 [https://calibre-ebook.com]', 'creationDate': "D:2017092014

## Edit the documents

In [28]:
for doc in docs_processed:
    doc.page_content = remove_repetitive_patterns(text = doc.page_content)

In [29]:
#remove all documents
remove(knowledge_vector_db, docstore_ids=None)
knowledge_vector_db.save_local(db_faiss_path)

## Save new docs to database

In [30]:
knowledge_vector_db = FAISS.from_documents(docs_processed, embedding_model)
# Save the new database, overwriting the old one
knowledge_vector_db.save_local(db_faiss_path)